## Leveraging Softmax Attention as a Replacement for Cosine Similarity in High-Dimensional Embedding Retrieval Systems (título fantasia)

<details>
<summary> <strong> Títulos possíveis </strong> </summary>

- An Attention-Driven Approach to Optimizing Embedding Retrieval in Vector Databases: A Softmax-Based Query-Key Mechanism for Enhanced Semantic Search
- Information Retrieval in High-Dimensional Vector Spaces: Leveraging Attention Mechanisms for Optimized Query-Embedding Projections
- Augmenting Vector Database Search via Scaled Dot-Product Attention: A Reformulation of Cosine Similarity in Hyperplane Embedding Spaces
- Softmax-Driven Query-Key Attention Models for Enhanced Similarity Matching in N-Dimensional Vectorial Manifolds
- Reformulating Similarity Search in Vector Databases: A Deep Dive into Attention-Driven Query Optimization
- Scaling Embedding-Based Information Retrieval with Attention Mechanisms: A Computational Approach to Vector Space Search
- Softmax Attention as a Replacement for Cosine Similarity in High-Dimensional Embedding Retrieval Systems
- Rearchitecting Vector Database Queries: Leveraging Attention Models for Enhanced Information Retrieval and Efficiency
- Optimizing Query Retrieval in Vector Databases: A Computational Model for Attention-Based Embedding Matching

</details>

GPT Description:

In this paper, we propose a novel method for optimizing information retrieval from vector databases using attention mechanisms, replacing traditional cosine similarity-based approaches. Our focus is on improving the efficiency and accuracy of query matching in large-scale embedding spaces, particularly in the context of retrieval-augmented generation (RAG) systems, where embeddings play a pivotal role in linking queries to relevant documents or knowledge units. By integrating a scaled dot-product attention mechanism into the retrieval process, we enhance the capacity of the system to prioritize more semantically relevant embeddings during search operations. This approach redefines the way queries interact with embeddings in a vector space, treating the retrieval task as an attention-based alignment problem. The attention mechanism dynamically weighs the importance of different dimensions in the query and embedding representations, ultimately improving the precision of retrieved results.

## To-do
Implementação em código / design de experimento
- [X] Implementar o attention (scaled dot-product attention mechanism)
- [X] Otimizar o mecanismo de atenção
- [X] Implementar o vetorial DB (att-based)
- [X] Vector DB com similaridade de cosseno para comparação
- [X] Vector DB com distância euclidiana para comparação
- [X] Vector DB com distância Manhattan para comparação
- [ ] Implementar [Multidimensional Comparisons: Dimension Insensitive Euclidean Metric](https://arxiv.org/pdf/2407.08623) (novo algoritmo que propõe uma substituição de  similaridade de cosseno)
- [ ] Replicar os testes do paper do DIEM, porém utilizando attention
- [ ] Organizar um benchmarking com vários datasets e desempenhar Needle in a Haystack em vários tamanhos (4k, 8k, 16k,..., 1000k) - padronizar as queries em todas as métricas de avaliação
- [ ] Rodar todos os modelos pelo menos 2000x para avaliar os resultados e tirar medidas de variação (std deviation e etc)
- [ ] Definir as métrica de avaliação para o retrieval

Benchmark e avaliação
- [Open Compass benchmarking](https://opencompass.readthedocs.io/en/latest/advanced_guides/needleinahaystack_eval.html)

Possíveis perguntas de pesquisa
- How can attention mechanisms improve the precision and recall of retrieval systems compared to traditional cosine similarity in high-dimensional embedding spaces?
- What are the computational trade-offs between using attention-based mechanisms and cosine similarity for large-scale vector database searches?
- How does using attention mechanisms impact the semantic alignment between query vectors and document embeddings compared to static similarity measures like cosine similarity?
- In what scenarios do attention-based retrieval mechanisms outperform cosine similarity in terms of context-sensitive search results?
- Can attention-based retrieval methods dynamically adjust the relevance of embeddings to better reflect changing contextual priorities in real-time search queries?
- What are the effects of integrating attention-based mechanisms with traditional vector search systems in terms of speed, accuracy, and flexibility?
- Can attention-based models effectively bridge the gap between query embeddings and document embeddings in low-resource or multilingual contexts where cosine similarity struggles?



## Setup
- Instalação de pacotes
- Imports
- Definição dos modelos-base de avaliação
- Definição do nano-dataset de teste para validar a ideia

In [1]:
%%capture
!pip install optuna
!pip install "opencompass[full]"

In [2]:
import os
import time
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import optuna as opt
import opencompass as oc

import torch
import torch.nn as nn
import transformers
from transformers import BertModel, BertTokenizer

In [3]:
%%capture
# Definindo o modelo utilizado
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [4]:
# Example sentences (use real embeddings in your use case)
sentences = [
    "The First Industrial Revolution began in Britain in the late 18th century.",
    "The invention of the steam engine was a key development during the First Industrial Revolution.",
    "Factories and mechanized production transformed industries like textiles and iron.",
    "The First Industrial Revolution marked a shift from manual labor to machine-based manufacturing.",
    "Railways expanded rapidly during the First Industrial Revolution, connecting distant regions.",
    "Steam power was used to run factories, locomotives, and ships, accelerating global trade.",
    "The Industrial Revolution had a profound impact on urbanization and the growth of cities.",
    "New inventions like the spinning jenny and the power loom revolutionized textile production.",
    "Child labor and poor working conditions were common in early industrial factories.",
    "The First Industrial Revolution laid the foundation for modern economies and industrial practices.",
    "Bananananananananananananodnsaubdyiqwofnquohr. What were the main inventions during the First Industrial Revolution?",
    "What were the main inventions during the First Industrial Revolution?",
    "Pizza barbecue paella"
]

query_sentence = "What were the main inventions during the First Industrial Revolution?" # Defining a query sentence

## Base functions

In [5]:
# Function to get BERT embedding for a sentence
def get_embedding(text, model, tokenizer):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]  # Use CLS token's embedding
    return embedding

In [15]:
def training(model, X, y, epochs):
    criterion = nn.Adam()  # You could use other losses like contrastive or triplet loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0017)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        output, _ = model(X)
        loss = criterion(output, y)  # Compare model output to target labels

        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

## Cosine DB

$ \cos({θ}) = \frac{|a^T . b|}{||a||.||b||}$

## ⚠️ Corrigir para usar o tokenizer do BERT

> Implementação retirada de [Building a Vector Database from Scratch in Python](https://medium.com/@vidiptvashist/building-a-vector-database-from-scratch-in-python-6bd683ba5171) por Vidipt Vashist


In [7]:
# VectorStore class to store BERT embeddings
class VectorStore_CS:
    def __init__(self):
        self.vector_data = {}  # A dictionary to store vectors

    def add_vector(self, vector_id, vector):
        """
        Add a vector to the store.
        Args:
            vector_id (str or int): A unique identifier for the vector.
            vector (numpy.ndarray): The vector data to be stored.
        """
        self.vector_data[vector_id] = vector

    def find_similar_vectors(self, query_vector, num_results=5):
        """
        Find similar vectors to the query vector using brute-force cosine similarity.
        Args:
            query_vector (numpy.ndarray): The query vector for similarity search.
            num_results (int): The number of similar vectors to return.
        Returns:
            list: A list of (vector_id, similarity_score) tuples for the most similar vectors.
        """
        query_vector = np.squeeze(query_vector)  # Remove extra dimensions
        results = []

        for vector_id, vector in self.vector_data.items():
            vector = np.squeeze(vector)  # Remove extra dimensions
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            results.append((vector_id, similarity))

        # Sort by similarity in descending order
        results.sort(key=lambda x: x[1], reverse=True)

        # Return the top N results
        return results[:num_results]


In [8]:
# Establish a VectorStore instance
vector_store = VectorStore_CS()

# Generate and store BERT embeddings in VectorStore
for sentence in sentences:
    embedding = get_embedding(sentence, model, tokenizer)
    vector_store.add_vector(sentence, embedding)

# Generate BERT embedding for the query sentence
query_vector = get_embedding(query_sentence, model, tokenizer)

# Find similar sentences using BERT embeddings
similar_sentences = vector_store.find_similar_vectors(query_vector, num_results=12)

# Display similar sentences
print("Query Sentence:", query_sentence)
print("Similar Sentences:")

for sentence, similarity in similar_sentences:
    print(f"{sentence}: Similarity = {similarity:.4f}")

Query Sentence: What were the main inventions during the First Industrial Revolution?
Similar Sentences:
What were the main inventions during the First Industrial Revolution?: Similarity = 1.0000
Bananananananananananananodnsaubdyiqwofnquohr. What were the main inventions during the First Industrial Revolution?: Similarity = 0.9137
The First Industrial Revolution laid the foundation for modern economies and industrial practices.: Similarity = 0.8561
The invention of the steam engine was a key development during the First Industrial Revolution.: Similarity = 0.8474
Child labor and poor working conditions were common in early industrial factories.: Similarity = 0.8322
Pizza barbecue paella: Similarity = 0.8299
The First Industrial Revolution marked a shift from manual labor to machine-based manufacturing.: Similarity = 0.8276
The First Industrial Revolution began in Britain in the late 18th century.: Similarity = 0.7919
The Industrial Revolution had a profound impact on urbanization and 

## Euclidian DB

$ d(x,y) = \sqrt{\sum_{k=1}^{n}(x_k - y_k)^2} $

In [9]:
class VectorStore_Euc:
    def __init__(self):
        self.vector_data = {}

    def add_vector(self, vector_id, vector):
        self.vector_data[vector_id] = vector

    def find_similar_vectors(self, query_vector, num_results: int):
      query_vector = np.squeeze(query_vector)  # Remove extra dimensions
      results = []

      for vector_id, vector in self.vector_data.items():
          vector = np.squeeze(vector)  # Remove extra dimensions
          euclidian = np.linalg.norm(query_vector - vector)
          results.append((vector_id, euclidian))

      # Sort by similarity in ascending order
      results.sort(key=lambda x: x[1], reverse=False)

      # Return the top N results
      return results[:num_results]

In [10]:
# Establish a VectorStore instance
vector_store_Euc = VectorStore_Euc()

# Generate and store BERT embeddings in VectorStore
for sentence in sentences:
    embedding = get_embedding(sentence, model, tokenizer)
    vector_store_Euc.add_vector(sentence, embedding)

# Generate BERT embedding for the query sentence
query_vector = get_embedding(query_sentence, model, tokenizer)

# Find similar sentences using BERT embeddings
similar_sentences = vector_store_Euc.find_similar_vectors(query_vector, num_results=12)

# Display similar sentences
print("Query Sentence:", query_sentence)
print("Similar Sentences:")

for sentence, similarity in similar_sentences:
    print(f"{sentence}: Similarity = {similarity:.4f}")

Query Sentence: What were the main inventions during the First Industrial Revolution?
Similar Sentences:
What were the main inventions during the First Industrial Revolution?: Similarity = 0.0000
Bananananananananananananodnsaubdyiqwofnquohr. What were the main inventions during the First Industrial Revolution?: Similarity = 6.0866
The First Industrial Revolution laid the foundation for modern economies and industrial practices.: Similarity = 8.2074
The invention of the steam engine was a key development during the First Industrial Revolution.: Similarity = 8.3710
Pizza barbecue paella: Similarity = 8.5552
Child labor and poor working conditions were common in early industrial factories.: Similarity = 8.7006
The First Industrial Revolution marked a shift from manual labor to machine-based manufacturing.: Similarity = 9.0159
The First Industrial Revolution began in Britain in the late 18th century.: Similarity = 9.9155
The Industrial Revolution had a profound impact on urbanization and 

## Manhattan DB (tá com algum bug)

$ d(x,y) = \sqrt{\sum_{k=1}^{n}|x_k - y_k|} $

In [11]:
class VectorStore_Man:
    def __init__(self):
        self.vector_data = {}

    def add_vector(self, vector_id, vector):
        self.vector_data[vector_id] = vector

    def find_similar_vectors(self, query_vector, num_results: int):
      query_vector = np.squeeze(query_vector)  # Remove extra dimensions
      results = []

      for vector_id, vector in self.vector_data.items():
          vector = np.squeeze(vector)  # Remove extra dimensions
          manhattan = np.absolute(query_vector - vector)
          results.append((vector_id, manhattan))

      ## Sort by similarity in ascending order
      results.sort(key=lambda x: x[1], reverse=False)

      # Return the top N results
      return results[:num_results]

In [12]:
# Establish a VectorStore instance
vector_store_Man = VectorStore_Man()

# Generate and store BERT embeddings in VectorStore
for sentence in sentences:
    embedding = get_embedding(sentence, model, tokenizer)
    vector_store_Man.add_vector(sentence, embedding)


# Generate BERT embedding for the query sentence
query_vector = get_embedding(query_sentence, model, tokenizer)


# Find similar sentences using BERT embeddings
similar_sentences = vector_store_Man.find_similar_vectors(query_vector, 12)


# Display similar sentences
print("Query Sentence:", query_sentence)
print("Similar Sentences:")

for sentence, similarity in similar_sentences:
    print(f"{sentence}: Similarity = {similarity:.4f}")

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

## Attention DB
$ Attention(Q,K,V) = softmax(\frac{QK^T}{ \sqrt{d_k}})V$


### Scaled dot-product attention mechanism

In [16]:
# Attention Model Definition
class AttentionModel(nn.Module):
    def __init__(self, emb_dim):
        super(AttentionModel, self).__init__()
        self.W_Q = nn.Linear(emb_dim, emb_dim)
        self.W_K = nn.Linear(emb_dim, emb_dim)
        self.W_V = nn.Linear(emb_dim, emb_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, query_embedding, sentence_embeddings):
        Q = self.W_Q(query_embedding)
        K = self.W_K(sentence_embeddings)
        V = self.W_V(sentence_embeddings)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(K.shape[-1], dtype=torch.float32))
        attention_weights = self.softmax(scores)
        output = torch.matmul(attention_weights, V)
        return output, attention_weights

### Query in Att Vector DB

In [17]:
# Function to query vector DB using attention mechanism
def query_vector_db(query, attention_model, vector_db, tokenizer, bert_model):
    # Get the query embedding
    query_embedding = get_embedding(query, bert_model, tokenizer)
    query_embedding = query_embedding.float().unsqueeze(0)  # Add batch dimension and ensure it's float32

    # Apply attention mechanism between query and stored embeddings
    db_tensor = vector_db.float()  # Ensure embeddings are float32
    output, attention_weights = attention_model(query_embedding, db_tensor)  # Pass both query and sentence embeddings

    return output, attention_weights

In [18]:
# Function to evaluate retrieval using attention mechanism
def evaluate_retrieval(attention_model, query, vector_db, tokenizer, bert_model):
    output, attention_weights = query_vector_db(query, attention_model, vector_db, tokenizer, bert_model)

    # Cosine similarity for comparison
    query_embedding = get_embedding(query, bert_model, tokenizer)
    cosine_similarities = torch.cosine_similarity(query_embedding.float(), vector_db.float())

    print("Attention-based Retrieval Output:", output)
    print("\nCosine Similarities:", cosine_similarities)


### Vector Store

In [19]:
# VectorStore class to store and retrieve embeddings
class VectorStore_Att:
    def __init__(self):
        self.vector_data = {}

    def add_vector(self, vector_id, vector):
        self.vector_data[vector_id] = vector

    def get_vectors(self):
        return self.vector_data

# Create a vector store and store sentence embeddings
vector_store_att = VectorStore_Att()

### Retrieval Evaluation

In [20]:
# Function to evaluate and display retrieval
def evaluate_retrieval(attention_model, query, vector_db, tokenizer, bert_model, sentences):
    output, attention_weights = query_vector_db(query, attention_model, vector_db, tokenizer, bert_model)

    # Cosine similarity for comparison
    query_embedding = get_embedding(query, bert_model, tokenizer)
    cosine_similarities = torch.cosine_similarity(query_embedding.float(), vector_db.float())

    # Sort sentences by similarity score (Cosine Similarity)
    sorted_indices = torch.argsort(cosine_similarities, descending=True)

    print(f"Query Sentence: {query}\n")
    print("Similar Sentences:")
    for idx in sorted_indices:
        sentence = sentences[idx]
        similarity = cosine_similarities[idx].item()
        print(f"{sentence}: Similarity = {similarity:.4f}")

In [21]:
# Generate embeddings for each sentence
embeddings = torch.cat([get_embedding(sentence, model, tokenizer) for sentence in sentences])

# Initialize the Attention Model
embed_dim = embeddings.shape[1]  # Dimensionality of BERT embeddings (768)
attention_model = AttentionModel(embed_dim)

evaluate_retrieval(attention_model, query_sentence, embeddings, tokenizer, model, sentences)

Query Sentence: What were the main inventions during the First Industrial Revolution?

Similar Sentences:
What were the main inventions during the First Industrial Revolution?: Similarity = 1.0000
Bananananananananananananodnsaubdyiqwofnquohr. What were the main inventions during the First Industrial Revolution?: Similarity = 0.9137
The First Industrial Revolution laid the foundation for modern economies and industrial practices.: Similarity = 0.8561
The invention of the steam engine was a key development during the First Industrial Revolution.: Similarity = 0.8474
Child labor and poor working conditions were common in early industrial factories.: Similarity = 0.8322
Pizza barbecue paella: Similarity = 0.8299
The First Industrial Revolution marked a shift from manual labor to machine-based manufacturing.: Similarity = 0.8276
The First Industrial Revolution began in Britain in the late 18th century.: Similarity = 0.7919
The Industrial Revolution had a profound impact on urbanization and

## DIEM DB

$DIEM = \frac{v_M - v_m}{\sigma(n)^2} \left( \sqrt{\sum_{i=1}^{n} (a_i - b_i)^2} - \mathbb{E}[d(n)] \right)$
- ${v_M}$ é o valor máximo que os elementos no vetor podem assumir;
- $v_m$ é o valor mínimo que os elementos do vetor podem assumir;
- $\sigma({n})^2$ é a variância da distância euclidiana para uma dada dimensão $n$;
- $a_i$ e $b_i$ são os _i-ésimos_ elementos de $\vec{a}$ e $\vec{b}$, respectivamente;
- $n$ é o número de dimensões nos vetores comparados;
- $\mathbb{E}[d(n)]$ é o valor esperado da distância euclidiana entre quaisquer dois vetores aleatórios $a, b \sim U(vm, vM)$ para uma dada dimensão $n$.

<!--
Calculando $\mathbb{E}[d(n)]$

Dada a definição do limite superior do valor que $\mathbb{E}[d(n)]$ pode assumir, definido pela desigualdade de Jensen e pela LOTUS (Law of the unconscious statistician), o limite superior de $\mathbb{E}[d(n)]$ é dado por:

$\mathbb{E}[d(n)] \leq \sqrt{\frac{n}{6}} \times (v_M - v_m), ∀  n \gt 2$, o limite superior é igual à mediana
-->

- $DIEM_{min} = \frac{-v_M - v_m}{\sigma(n)^2} \mathbb{E}[d(n)]$

- $DIEM_{max} = \frac{v_M - v_n}{\sigma(n)^2} \sqrt{n} (v_M - v_m) - \mathbb{E}[d(n)] $

### Calculando o desvio padrão $\sigma(n)$

In [ ]:
def calcula_sigma_n(
    n: int,
    vM: float,
    vm:float,
    num_samples: int =10000): -> float

    """
    Calcula o desvio padrão da distância euclidiana para a dimensão n
    usando simulação.

    Args:
      n: Número de dimensões.
      vM: Valor máximo dos elementos do vetor.
      vm: Valor mínimo dos elementos do vetor.
      num_samples: Número de amostras para a simulação (opcional).

    Returns:
      O desvio padrão da distância euclidiana para a dimensão n.
    """
    vetores_a = np.random.uniform(vm, vM, size=(num_samples, n))
    vetores_b = np.random.uniform(vm, vM, size=(num_samples, n))
    distancias = np.linalg.norm(vetores_a - vetores_b, axis=1)
    return np.std(distancias)

### Calculando $DIEM$

In [ ]:
def diem(
    a: np.ndarray,
    b: np.ndarray,
    vM: float,
    vm: float):
    """
    Calcula o DIEM entre dois vetores a e b.

    Args:
      a: O primeiro vetor.
      b: O segundo vetor.
      vM: Valor máximo dos elementos do vetor.
      vm: Valor mínimo dos elementos do vetor.

    Returns:
      O valor DIEM entre os vetores a e b.
    """
    n = len(a)
    sigma_n = calcula_sigma_n(n, vM, vm)
    distancia_euclidiana = np.linalg.norm(a - b)
    # Cálculo do DIEM usando a Equação 13 da
    diem_value = (vM - vm) / (sigma_n ** 2) * (distancia_euclidiana - np.sqrt(n * ((vM - vm) ** 2) / 6))
    return diem_value

In [ ]:
class VectorStore_DIEM:
    def __init__(self, vM, vm):
        self.vector_data = {}
        self.vM = vM
        self.vm = vm

    def add_vector(self, vector_id, vector):
        """Armazena um vetor na base de dados."""
        self.vector_data[vector_id] = vector

    def find_similar_vectors(self, query_vector, num_results=5):
        """Encontra os vetores mais similares usando a métrica DIEM."""
        query_vector = np.squeeze(query_vector)

        results = []
        for vector_id, vector in self.vector_data.items():
            vector = np.squeeze(vector)

            # Usando DIEM para calcular a similaridade
            similarity = diem(query_vector, vector, self.vM, self.vm)
            results.append((vector_id, similarity))

        # Ordenando por similaridade DIEM (menor distância = mais similar)
        results.sort(key=lambda x: x[1])
        return results[:num_results]

In [ ]:
# Initialize the vector store with DIEM settings
vector_store_DIEM = VectorStore_DIEM(vM=1.0, vm=0.0)

# Assuming get_embedding function and BERT model are already defined
# Generate and store BERT embeddings in VectorStore
for sentence in sentences:
    embedding = get_embedding(sentence, model, tokenizer)
    vector_store_DIEM.add_vector(sentence, embedding)

# Generate BERT embedding for the query sentence
query_sentence = "What were the main inventions during the First Industrial Revolution?"
query_vector = get_embedding(query_sentence, model, tokenizer)

# Find similar sentences using BERT embeddings
similar_sentences = vector_store_DIEM.find_similar_vectors(query_vector, num_results=12)

# Display similar sentences
print("Query Sentence:", query_sentence)
print("Similar Sentences:")

for sentence, similarity in similar_sentences:
    print(f"{sentence}: Similarity = {similarity:.4f}")

## Global evaluator
Implementando um sistema de avaliação global comum a todos os algoritmos para execução de Needle in a haystack e para isso, precisamos:

- [ ] Refazer o banco vetorial, implementando um único banco global e executar os testes apenas modificando o mecanismo de busca;
- [ ] Extrair os dados de consumo computacional durante a execução das buscas;
- [ ] Definir os modelos que executaremos a busca;
- [ ] Selecionar o dataset de Needle in a Haystack para utilizar;
- [ ] Automaticamente coletar os dados relacionados à avaliação do desempenho do modelo;


In [ ]:
class VectorStore:
  def __init__(self):


## Testing the OpenCompass